# Initialization

### Loading dependencies

In [ ]:
import os
import random
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

tfk = tf.keras
tfkl = tf.keras.layers

import visualkeras

print(tf.__version__)

### Set seed for reproducibility

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Suppress warnings

In [ ]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Plants dataset

In [ ]:
dataset_dir = './datasetTEST'
training_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'val')
test_dir = os.path.join(dataset_dir, 'test')

In [ ]:
# Plot example images from dataset
labels = [
          'Species1',   # 0
          'Species2',   # 1
          'Species3',   # 2
          'Species4',   # 3
          'Species5',   # 4
          'Species6',   # 5
          'Species7',   # 6
          'Species8',   # 7
          ]

### Models metadata

In [ ]:
# Input Parameters
img_w = 96
img_h = 96
input_shape = (96, 96, 3)
classes = 8

# Training Parameters
epochs = 200
batch_size = 32
learning_rate_high = 1e-2
learning_rate_low = 1e-5

# Earlystopping Parameters
early_stopping = True
patience_epochs = 10

# Data Augmentation
apply_augmentation = True

In [ ]:
from  sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(train_gen.classes), 
            y=train_gen.classes)
# Keras requires a dictionary
class_weights = {i : class_weights[i] for i in range(len(class_weights))}

# Data Augmentation

In [ ]:
from PIL import Image, ImageEnhance


def adjust_brightness(im, factor):
    im = tf.keras.utils.array_to_img(im)
    #image brightness enhancer
    enhancer = ImageEnhance.Brightness(im)
    
    im_output = enhancer.enhance(factor)
    im_output = tf.keras.utils.img_to_array(im_output)
    return im_output

In [ ]:
def preprocessing(image_gen):
    image_saturated = tf.image.adjust_saturation(image_gen,2.5)
    image_brightness = adjust_brightness(image_saturated, 1.2)
    
    return image_brightness

In [ ]:
valid_data_gen = ImageDataGenerator(rescale = 1./255, preprocessing_function = preprocessing)
test_data_gen = ImageDataGenerator(rescale = 1./255, preprocessing_function = preprocessing)

valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(img_w,img_h),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=True,
                                               seed=seed)
test_gen = test_data_gen.flow_from_directory(directory=test_dir,
                                             target_size=(img_w,img_h),
                                             color_mode='rgb',
                                             classes=None, # can be set to labels
                                             class_mode='categorical',
                                             batch_size=batch_size,
                                             shuffle=True,
                                             seed=seed)

In [ ]:
# Create an instance of ImageDataGenerator with Data Augmentation
if apply_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=5,
                                        height_shift_range=5,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        #brightness_range=[0.5, 1.4],
                                        fill_mode='reflect',
                                        rescale= 1./255)  # rescale value is multiplied to the image
    print("AUGMENTED DATASET")
else:
    train_data_gen = ImageDataGenerator(rescale = 1./255, preprocessing_function = preprocessing)
    print("NON-AUGMENTED DATASET")

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                                target_size=(img_w,img_h),
                                                color_mode='rgb',
                                                classes=None, # can be set to labels
                                                class_mode='categorical',
                                                batch_size=batch_size,
                                                shuffle=True,
                                                seed=seed)

# CNN model

In [ ]:
def build_basic_model(input_shape):
    tf.random.set_seed(seed)

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    
    x = tfkl.Conv2D(
        filters = 25,
        kernel_size = 3,
        padding = 'same',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'conv1')(input_layer)
    x = tfkl.ReLU()(x)
    x = tfkl.MaxPooling2D(name='mp1')(x)

    x = tfkl.Conv2D(
        filters = 50,
        kernel_size = 3,
        padding = 'same',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'conv2')(x)
    x = tfkl.ReLU()(x)
    x = tfkl.MaxPooling2D(name='mp2')(x)

    x = tfkl.Conv2D(
        filters = 100,
        kernel_size = 3,
        padding = 'same',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'conv3')(x)
    x = tfkl.ReLU()(x)
    x = tfkl.MaxPooling2D(name='mp3')(x)

    x = tfkl.GlobalAveragePooling2D(name='GAP_Layer')(x)
    x = Dropout(0.3, seed=seed, name='classifier_dropout_1')(x)
    x = tfkl.Dense(
        units = 256,
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'classifier')(x)
    x = tfkl.ReLU()(x)
    x = tfkl.Dropout(0.3, seed=seed, name='classifier_dropout_2')(x)

    output_layer = tfkl.Dense(
        units = classes, 
        activation = 'softmax', 
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs = input_layer, outputs = output_layer, name = 'BasicModel')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
basic_model = build_basic_model(input_shape)

In [ ]:
basic_model.summary()

In [ ]:
# Learning Rate Scheduler
def scheduler(epoch, lr):
   if epoch < 100:
     return lr
   else:
     return lr * tf.math.exp(-0.1)

In [ ]:
# Utility function to create folders and callbacks for training
def create_folders_and_callbacks(model_name) :
    callbacks = []

    # Early Stopping -----------------------------------------------------
    if early_stopping:
        es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto', patience=patience_epochs, restore_best_weights=True)
        callbacks.append(es_callback)

    # Checkpointer
    checkpointer = tfk.callbacks.ModelCheckpoint(filepath='./checkpoint/weights_basic_model.h5', verbose=1, 
                                    save_best_only=True, monitor = "val_accuracy", mode = "auto",)
    callbacks.append(checkpointer)
    
    # Learning Rate Scheduler --------------------------------------------
    LRS_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
    callbacks.append(LRS_callback)
    
    return callbacks

In [ ]:
callbacks = create_folders_and_callbacks(model_name='CustomCNN')

In [ ]:
# Compile the model
basic_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

basic_history = basic_model.fit(
    x = train_gen,
    class_weight = class_weights,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = callbacks,
).history

# Testing

In [ ]:
valid_data_gen = ImageDataGenerator(rescale = 1./255, preprocessing_function = preprocessing)
test_data_gen = ImageDataGenerator(rescale = 1./255, preprocessing_function = preprocessing)

valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(img_w,img_h),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=batch_size,
                                               shuffle=False,
                                               seed=seed)
test_gen = test_data_gen.flow_from_directory(directory=test_dir,
                                             target_size=(img_w,img_h),
                                             color_mode='rgb',
                                             classes=None, # can be set to labels
                                             class_mode='categorical',
                                             batch_size=batch_size,
                                             shuffle=False,
                                             seed=seed)

In [ ]:
# Print confusion matrix on test set
test_steps_per_epoch = np.math.ceil(test_gen.samples / test_gen.batch_size)

# Evaluate on test
predictions = basic_model.predict(test_gen, steps=test_steps_per_epoch)

# Get most likely classes
predicted_classes = np.argmax(predictions, axis=-1)

# Get true classes
true_classes = test_gen.classes
class_labels = list(test_gen.class_indices.keys())

# Compute the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Compute the classification metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='macro')
recall = recall_score(true_classes, predicted_classes, average='macro')
f1 = f1_score(true_classes, predicted_classes, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(7,5))
sns.heatmap(cm.T, xticklabels=list(class_labels), yticklabels=class_labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
# Print Confusion Matrix and Classification Report (Precision, Recall, and F1-score) on the validation set
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

Y_prediction = basic_model.predict_generator(valid_gen, len(valid_gen))
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = valid_gen.classes
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
class_report = classification_report(Y_true, Y_pred_classes, 
                                     target_names=valid_gen.class_indices.keys())  # target_names must be ordered depending on the class labels
print('Confusion Matrix:')
print(confusion_mtx)
print()
print('Classification Report:')
print(class_report)

In [ ]:
# Predict basic_model
print("Basic model: ")
model_test_metrics = basic_model.evaluate(test_gen, return_dict=True)

In [ ]:
# Plot loss
plt.figure(figsize=(15,5))

plt.plot(basic_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(basic_history['val_loss'], label='Basic model', alpha=.8, color='#4D61E2')

plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)


# Plot accuracy
plt.figure(figsize=(15,5))
plt.plot(basic_history['accuracy'], alpha=.3, color='#77bbff', linestyle='--')
plt.plot(basic_history['val_accuracy'], label='Basic model', alpha=.8, color='#77bbff')


plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
basic_model.save('basic_model_saturation')